In [47]:
# !pip install openpyxl --upgrade

In [48]:
import pandas as pd
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [49]:
finance_data = pd.read_excel ('./Data_finance_lab3.xlsx')
finance_data.head(5)

,APP_ID,DISBURSALDATE,REPORT_DATE,TERM_TMP,PRODUCT_NAME,LOANAMOUNT_NOT_INSURANCE,INSURANCE_FEE,LA,EFF_RATE,TERM,...,PERSONAL_INCOME,ADDITIONAL_INCOME,PERSONAL_EXPENSE,FAMILY_INCOME,FAMILY_EXPENSE,MARITAL_STATUS,EDUCATION,City,ADDRESS_CURRENT_CITY,PROVINCE_CURRENT
0,16499318,2017-10-10,2018-10-26,12,FC UP DSA TS SCORE 2 NEW- 762,18000000,990000,18990000,45.0,24,...,9000000,0,9600000,20000000,5000000,Other,High School,Tỉnh Yên Bái,Yên Bình,Tỉnh Yên Bái
1,16490594,2017-10-10,2018-10-26,12,FC UP L.G 2 TSA SCORE NEW- 617,10000000,550000,10550000,44.5,12,...,5500000,0,7200000,15000000,4000000,Married,High School,Tỉnh Quảng Ninh,Đông Triều,Tỉnh Quảng Ninh
2,16910227,2017-10-28,2018-10-26,11,FC UP CAT B NEW- 610,25000000,1375000,26375000,47.0,24,...,5182000,0,12000000,5200000,1000000,Single,Middle School,Tỉnh Bình Dương,Tân Uyên,Tỉnh Bình Dương
3,16355347,2017-10-04,2018-10-26,12,FC UP L.G 2 TSA SCORE NEW- 617,30000000,1650000,31650000,44.5,24,...,10000000,0,18000000,20000000,6000000,Married,Middle School,Tỉnh An Giang,Phú Tân,Tỉnh An Giang
4,16705974,2017-10-18,2018-10-26,12,FC UP L.G 2 TSA SCORE NEW- 617,20000000,1100000,21100000,44.5,18,...,7000000,0,12000000,20000000,7000000,Single,High School,Thành phố Hải Phòng,Vĩnh Bảo,Thành phố Hải Phòng


- giu cot: term_tmp, EFF_RATE, TERM, COUNT_TERM_RECEIPT, SUM_RECEIPT_AMT, GENDER, AGE, 'Number of Children', PERSONAL_INCOME, ADDITIONAL_INCOME, FAMILY_EXPENSE, INCOME (should be removed), MARTIAL STATUS, EDUCATION, RESULT_YES/NO
- bo cot: DISBURSALDATE,	REPORT_DATE, 
- LA = LOANAMOUNT_NOT_INSURANCE + INSURANCE_FEE

# Preprocess Dataset

In [50]:
finance_data.columns

Index(['APP_ID', 'DISBURSALDATE', 'REPORT_DATE', 'TERM_TMP', 'PRODUCT_NAME',
       'LOANAMOUNT_NOT_INSURANCE', 'INSURANCE_FEE', 'LA', 'EFF_RATE', 'TERM',
       'COUNT_TERM_RECEIPT', 'flag term', 'SUM_RECEIPT_AMT', 'Unnamed: 13',
       'flag loan amt', 'flag final', 'RESULT_YES/NO', 'EMI', 'GENDER', 'AGE',
       'Number of Children', 'PERSONAL_INCOME', 'ADDITIONAL_INCOME',
       'PERSONAL_EXPENSE', 'FAMILY_INCOME', 'FAMILY_EXPENSE', 'MARITAL_STATUS',
       'EDUCATION', 'City', 'ADDRESS_CURRENT_CITY', 'PROVINCE_CURRENT'],
      dtype='object')

In [51]:
# Retain necessary columns
columns_to_keep = ['TERM_TMP', 'EFF_RATE', 'TERM', 'COUNT_TERM_RECEIPT', 'SUM_RECEIPT_AMT', 'GENDER', 'AGE', 'Number of Children', 'PERSONAL_INCOME', 'ADDITIONAL_INCOME', 'FAMILY_EXPENSE', 'MARITAL_STATUS', 'EDUCATION', 'RESULT_YES/NO']
finance_data = finance_data[columns_to_keep]

In [52]:
finance_data['Number of Children'].isna().sum()

4200

In [53]:
# Convert categorical columns
label_encoders = {}
for col in ['GENDER', 'MARITAL_STATUS', 'EDUCATION','TERM_TMP','RESULT_YES/NO']:
    le = LabelEncoder()
    finance_data[col] = le.fit_transform(finance_data[col].astype(str))
    label_encoders[col] = le

In [54]:
# Handle missing values
finance_data['COUNT_TERM_RECEIPT'].fillna(0, inplace=True)
finance_data['SUM_RECEIPT_AMT'].fillna(0, inplace=True)
finance_data['Number of Children'].fillna(0, inplace=True)
# Other columns can be handled similarly based on context.

C:\Users\Bafo\AppData\Local\Temp\ipykernel_13724\3647737687.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  finance_data['COUNT_TERM_RECEIPT'].fillna(0, inplace=True)
C:\Users\Bafo\AppData\Local\Temp\ipykernel_13724\3647737687.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [55]:
numerical_cols = ['EFF_RATE',"Number of Children", "COUNT_TERM_RECEIPT", "SUM_RECEIPT_AMT", 'TERM', 'AGE', 'PERSONAL_INCOME', 'ADDITIONAL_INCOME', 'FAMILY_EXPENSE']
scaler = StandardScaler()
finance_data[numerical_cols] = scaler.fit_transform(finance_data[numerical_cols])


# Model

In [56]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Prepare data without the label for clustering
X = finance_data.drop('RESULT_YES/NO', axis=1)
y_true = finance_data['RESULT_YES/NO']  # True labels used only for evaluation

# Apply K-Means clustering with 2 clusters
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

# Retrieve cluster assignments
cluster_labels = kmeans.labels_

# Map clusters to the original labels
# We'll determine which cluster corresponds to 'YES' and which to 'NO'
# by checking the majority class in each cluster

# Convert cluster labels to match 0 and 1 labels of RESULT_YES/NO
cluster_to_label = {}
for cluster in [0, 1]:
    # Find the majority label in each cluster for mapping
    majority_label = y_true[cluster_labels == cluster].mode()[0]
    cluster_to_label[cluster] = majority_label

# Map clusters to predicted labels
y_pred = [cluster_to_label[cluster] for cluster in cluster_labels]

# Evaluate the clustering results
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Unsupervised Clustering Performance - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

# Print out examples of the clustering results
finance_data['Predicted_RESULT_YES/NO'] = y_pred
print("Sample of Predicted Labels:")
print(finance_data[['RESULT_YES/NO', 'Predicted_RESULT_YES/NO']].head(10))

Unsupervised Clustering Performance - Accuracy: 0.7272, Precision: 0.7272, Recall: 1.0, F1 Score: 0.842056507642427
Sample of Predicted Labels:
   RESULT_YES/NO  Predicted_RESULT_YES/NO
0              1                        1
1              1                        1
2              1                        1
3              0                        1
4              1                        1
5              1                        1
6              0                        1
7              1                        1
8              1                        1
9              1                        1


# Anomaly Detection

In [57]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

# Load and preprocess data (assuming 'data' is already preprocessed and loaded as per the previous steps)

# Prepare the dataset without 'RESULT_YES/NO' as a target column
X = finance_data.copy()

# Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)  # Set contamination level as per anomaly percentage
iso_forest.fit(X)
iso_anomalies = iso_forest.predict(X)

# Local Outlier Factor
lof = LocalOutlierFactor(contamination=0.05, n_neighbors=20)
lof_anomalies = lof.fit_predict(X)

# Extracting anomaly samples identified by each method
iso_anomalous_samples = X[iso_anomalies == -1]  # -1 indicates anomaly in Isolation Forest
lof_anomalous_samples = X[lof_anomalies == -1]  # -1 indicates anomaly in LOF

print("Anomalies detected by Isolation Forest:")
print(iso_anomalous_samples)

print("\nAnomalies detected by Local Outlier Factor:")
print(lof_anomalous_samples)

Anomalies detected by Isolation Forest:
      TERM_TMP  EFF_RATE      TERM  COUNT_TERM_RECEIPT  SUM_RECEIPT_AMT  \
6            1 -0.155859  1.678836           -2.357648        -1.327373   
22           1 -0.084957 -0.342072            1.330610         3.815929   
41           1 -0.155859  1.678836           -1.790224         2.105441   
62           1 -0.155859  0.331564            0.763186         2.033419   
75           1  0.009578  2.352472           -0.087951         0.765432   
...        ...       ...       ...                 ...              ...   
4936         1 -1.738850 -1.015709           -0.655375        -0.595578   
4943         1  2.278422 -1.015709           -0.655375        -1.150382   
4956         1 -1.738850  0.331564           -1.222800         3.155476   
4979         1 -1.738850 -0.678891            0.195761        -0.239777   
4991         1  0.860394 -0.342072           -2.073936        -1.206720   

      GENDER       AGE  Number of Children  PERSONAL_INCOME

In [58]:
# Assuming 'iso_anomalous_samples' and 'lof_anomalous_samples' are DataFrames of anomalies detected by Isolation Forest and LOF respectively.

# Inverse scaling for numerical columns
numerical_cols = ['EFF_RATE',"Number of Children", "COUNT_TERM_RECEIPT", "SUM_RECEIPT_AMT", 'TERM', 'AGE', 'PERSONAL_INCOME', 'ADDITIONAL_INCOME', 'FAMILY_EXPENSE']
iso_anomalous_samples[numerical_cols] = scaler.inverse_transform(iso_anomalous_samples[numerical_cols])
lof_anomalous_samples[numerical_cols] = scaler.inverse_transform(lof_anomalous_samples[numerical_cols])

# Inverse encoding for categorical columns
for col, le in label_encoders.items():
    iso_anomalous_samples[col] = le.inverse_transform(iso_anomalous_samples[col].astype(int))
    lof_anomalous_samples[col] = le.inverse_transform(lof_anomalous_samples[col].astype(int))


C:\Users\Bafo\AppData\Local\Temp\ipykernel_13724\2822861248.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iso_anomalous_samples[numerical_cols] = scaler.inverse_transform(iso_anomalous_samples[numerical_cols])
C:\Users\Bafo\AppData\Local\Temp\ipykernel_13724\2822861248.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lof_anomalous_samples[numerical_cols] = scaler.inverse_transform(lof_anomalous_samples[numerical_cols])
C:\Users\Bafo\AppData\Local\Temp\ipykernel_13724\2822861248.py:10: SettingWithCo

In [59]:
iso_anomalous_samples

,TERM_TMP,EFF_RATE,TERM,COUNT_TERM_RECEIPT,SUM_RECEIPT_AMT,GENDER,AGE,Number of Children,PERSONAL_INCOME,ADDITIONAL_INCOME,FAMILY_EXPENSE,MARITAL_STATUS,EDUCATION,RESULT_YES/NO,Predicted_RESULT_YES/NO
6,12,33.50,30.0,0.0,0.0,F,28.0,0.0,25000000.0,0.0,8000000.0,Divorced,College,0,1
22,12,35.00,12.0,13.0,50941380.0,M,50.0,0.0,20000000.0,0.0,5000000.0,Married,Basic School,1,1
41,12,33.50,30.0,2.0,34000000.0,M,26.0,0.0,25000000.0,0.0,5000000.0,Married,Working Cerfiticate,1,1
62,12,33.50,18.0,11.0,33286671.0,M,38.0,0.0,40000000.0,0.0,15000000.0,Married,High School,0,1
75,12,37.00,36.0,8.0,20728000.0,F,59.0,0.0,20000000.0,0.0,7000000.0,Married,High School,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4936,12,0.01,6.0,6.0,7248000.0,F,52.0,7.0,12000000.0,0.0,7000000.0,Married,Middle School,1,1
4943,12,85.00,6.0,6.0,1753000.0,F,49.0,3.0,5000000.0,0.0,5000000.0,Married,Middle School,1,1
4956,12,0.01,18.0,4.0,44399986.0,F,31.0,0.0,10000000.0,0.0,5000000.0,Other,Bachelor,1,1
4979,12,0.01,9.0,9.0,10772000.0,F,49.0,0.0,15000000.0,0.0,2000000.0,Divorced,Working Cerfiticate,1,1


In [61]:
lof_anomalous_samples

,TERM_TMP,EFF_RATE,TERM,COUNT_TERM_RECEIPT,SUM_RECEIPT_AMT,GENDER,AGE,Number of Children,PERSONAL_INCOME,ADDITIONAL_INCOME,FAMILY_EXPENSE,MARITAL_STATUS,EDUCATION,RESULT_YES/NO,Predicted_RESULT_YES/NO
5,12,37.00,18.0,12.0,13961737.0,M,99.0,0.0,5300000.0,0.0,4000000.0,Married,High School,1,1
22,12,35.00,12.0,13.0,50941380.0,M,50.0,0.0,20000000.0,0.0,5000000.0,Married,Basic School,1,1
41,12,33.50,30.0,2.0,34000000.0,M,26.0,0.0,25000000.0,0.0,5000000.0,Married,Working Cerfiticate,1,1
74,12,35.00,18.0,13.0,29874000.0,F,41.0,0.0,15000000.0,0.0,1000000.0,Single,High School,1,1
84,12,47.00,36.0,9.0,5071350.0,F,21.0,0.0,4114000.0,0.0,800000.0,Single,Master Degree,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4956,12,0.01,18.0,4.0,44399986.0,F,31.0,0.0,10000000.0,0.0,5000000.0,Other,Bachelor,1,1
4973,12,0.01,6.0,6.0,4337000.0,F,50.0,0.0,7000000.0,0.0,8000000.0,Single,High School,1,1
4975,12,0.01,4.0,4.0,3240000.0,F,41.0,0.0,20000000.0,0.0,2000000.0,Married,Middle School,1,1
4979,12,0.01,9.0,9.0,10772000.0,F,49.0,0.0,15000000.0,0.0,2000000.0,Divorced,Working Cerfiticate,1,1
